### Memoization Examples

Save invariant results for otherwise expensive calculations.

See [Memoization: wikipedia](https://en.wikipedia.org/wiki/Memoization)

See [Fibonacci: wikipedia](https://en.wikipedia.org/wiki/Fibonacci_number)

In [1]:
debugging = False
debugging2 = False
#debugging = True

logging = True

def dbg(f, *args):
    if debugging:
        print(('  DBG:' + f).format(*args))
        
def dbg2(f, *args):
    if debugging2:
        print(('  DBG2:' + f).format(*args))

def log(f, *args):
    if logging:
        print((f).format(*args))
        
def logError(f, *args):
    if logging:
        print(('*** ERROR:' + f).format(*args))
        
def className(instance):
    return type(instance).__name__

### The TestSet Mechanism

In [16]:
# %load TestHarness.py
class TestCase(object):
    def __init__(self, name, method, inputs, expected, catchExceptions=False):
        self.name = name
        self.method = method
        self.inputs = inputs
        self.expected = expected
        self.catchExceptions = catchExceptions
        
    def run(self):
        if self.catchExceptions:
            try:
                return self.method(*self.inputs)
            except Exception as x:
                return x
        else:
                return self.method(*self.inputs)

import time
from datetime import timedelta

class TestSet(object):
    def __init__(self, cases):
        self.cases = cases
    
    def run_tests(self, repeat=1):
        count = 0
        errors = 0
        total_time = 0
        for case in self.cases:
            count += 1
            start_time = time.time()
            for iteration in range(repeat):
                dbg2("*** Running '{0}' iteration {1}", case.name, iteration+1)
                result = case.run()
            elapsed_time = time.time() - start_time
            total_time += elapsed_time
            if callable(case.expected):
                if not case.expected(result):
                    errors += 1
                    logError("Test {0} failed. Returned {1}", case.name, result)
            elif result != case.expected:
                errors += 1
                logError('Test {0} failed. Returned "{1}", expected "{2}"', case.name, result, case.expected)
        if errors:
            logError("Tests passed: {0}; Failures: {1}", count-errors, errors)
        else:
            log("All {0} tests passed.", count)
        repinfo = "" if repeat < 2 else " -- each case repeated {0} times.".format(repeat)
        log("Elapsed test time: {0}{1}", timedelta(seconds=total_time), repinfo)

### The Unit Under Test

In [3]:
def fib1(n):
    fib = fib1
    if n <= 0:
        return 0
    if n == 1:
        return 1
    return fib(n-1) + fib(n-2)

In [4]:
fibs = {} # Global memo table
def fib2(n):  
    fib = fib2
    if n in fibs: return fibs[n]
    if n <= 0:
        f = 0
    elif n == 1:
        f = 1
    else:
        f = fib(n-1) + fib(n-2)
    fibs[n] = f
    return f

In [5]:
def static_vars(**kwargs):
    def decorate(func):
        for k in kwargs:
            setattr(func, k, kwargs[k])
        return func
    return decorate

@static_vars(__fibMemos={})
def fib3(n):  
    func = fib3
    if n in func.__fibMemos:
        f = func.__fibMemos[n]
    else:
        if n <= 0:
            f = 0
        elif n == 1:
            f = 1
        else:
            f = func(n-1) + func(n-2)
        func.__fibMemos[n] = f
    return f

In [6]:
fib3(132)

1725375039079340637797070384

### The Test Cases

In [18]:
def simpletest(n):   
    return fib(n)

tset = [
           TestCase( '0', simpletest, [ 0 ],  0),
           TestCase( '1', simpletest, [ 1 ],  1),
           TestCase( '2', simpletest, [ 2 ],  1),
           TestCase( '3', simpletest, [ 3 ],  2),
           TestCase( '4', simpletest, [ 4 ],  3),
           TestCase( '5', simpletest, [ 5 ],  5),
           TestCase( '6', simpletest, [ 6 ],  8),
           TestCase( '7', simpletest, [ 7 ],  13),
           TestCase( '8', simpletest, [ 8 ],  21),
           TestCase( '9', simpletest, [ 9 ],  34),
           TestCase('10', simpletest, [ 10 ], 55),
           TestCase('20', simpletest, [ 20 ], 6765),
           TestCase('25', simpletest, [ 25 ], 75025),
           TestCase('30', simpletest, [ 30 ], 832040),
 #          TestCase('35', simpletest, [ 35 ], 9227465),
       ]

tester = TestSet(tset)

for fib in [fib1, fib2, fib3]:
    tester.run_tests(repeat=10)

All 14 tests passed.
Elapsed test time: 0:00:09.569891 -- each case repeated 10 times.
All 14 tests passed.
Elapsed test time: 0:00:00.000278 -- each case repeated 10 times.
All 14 tests passed.
Elapsed test time: 0:00:00.000301 -- each case repeated 10 times.


In [19]:
for fib in [fib2, fib3]:
    tester.run_tests(repeat=100000)

All 14 tests passed.
Elapsed test time: 0:00:02.682670 -- each case repeated 100000 times.
All 14 tests passed.
Elapsed test time: 0:00:03.231853 -- each case repeated 100000 times.


### Some Ad Hoc Tests

In [20]:
(.33333333333333333333333333333).as_integer_ratio()

(6004799503160661, 18014398509481984)

In [21]:
(1/3).as_integer_ratio()

(6004799503160661, 18014398509481984)

In [22]:
from fractions import Fraction
Fraction(0.333333333333333).limit_denominator()

Fraction(1, 3)

In [23]:
a = [1, 2, 3]
a.reverse()
a

[3, 2, 1]

In [31]:
join([x for x in reversed("ABC")])

NameError: name 'join' is not defined